In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)
print(tf.config.experimental.list_physical_devices('GPU'))

import random
import pickle
import spacy

2.2.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import sys
sys.path.append('..')
import ui_utils
import nlp_tools

In [3]:
train_df = pd.read_json('../input/security_100_coded.json', orient='records')

In [4]:
ui_utils.create_tag_columns(train_df).head()

,UID,Text,Tags,Tag_bar,Tag_lock,Tag_intercom,Tag_police,Tag_none,Tag_sign,Tag_doorkeeper,Tag_doorbell,Tag_camera,Tag_light,Tag_dog,Tag_irrelevant,Tag_gate,Tag_key
0,10000302,Ring Doorbell,doorbell,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,10000465,I trailed behind to get thru gate,gate,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,10001350,"Beautiful neighborhood, home is very well kept...",irrelevant,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,10001811,Locked gate with a pad lock,gate,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,10001877,No issues with access.,none,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [5]:
train_df = ui_utils.create_tag_columns(train_df)

In [6]:
train_df.head(30)

,UID,Text,Tags,Tag_bar,Tag_lock,Tag_intercom,Tag_police,Tag_none,Tag_sign,Tag_doorkeeper,Tag_doorbell,Tag_camera,Tag_light,Tag_dog,Tag_irrelevant,Tag_gate,Tag_key
0,10000302,Ring Doorbell,doorbell,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,10000465,I trailed behind to get thru gate,gate,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,10001350,"Beautiful neighborhood, home is very well kept...",irrelevant,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,10001811,Locked gate with a pad lock,gate,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,10001877,No issues with access.,none,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,10001963,Must be buzzed in with the intercom,intercom,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6,10002085,Deadbolt lock,lock,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,10002397,Many street lights and lampposts on the property.,light,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8,10003709,This might be a 2 family. There were 2 address...,irrelevant,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9,10003839,This apartment complex is gated. Not able to g...,gate,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [7]:
train_df.Text

0                                         Ring Doorbell
1                     I trailed behind to get thru gate
2     Beautiful neighborhood, home is very well kept...
3                           Locked gate with a pad lock
4                                No issues with access.
                            ...                        
95    Very poorly managed; park anywhere; people wal...
96    Sign on door says don’t ring doorbell, don’t m...
97                                 No security features
98                     Had video camera pointed to door
99    This is a large home on a dead end middle inco...
Name: Text, Length: 100, dtype: object

In [8]:
%%time
en_model = spacy.load('en_core_web_lg')

CPU times: user 3.8 s, sys: 283 ms, total: 4.08 s
Wall time: 4.07 s


In [9]:
def identify_connectors(document):
    """
    ---INPUT---
    document: a Doc() object processed by SpaCy Pipeline
    ---OUTPUT---
    connector_dict: a dictionary of connectors and their index
    """
    connector_dict = {
        'text':[],
        'pos':[],
        'index':[]
    }
    for index, token in enumerate(document):
        if token.pos_ in ['PUNCT', 'CCONJ', 'SCONJ'] or token.text.lower() == 'however':
            connector_dict['text'].append(token.text)
            connector_dict['pos'].append(token.pos_)
            connector_dict['index'].append(index)
    return connector_dict # dict of index


def split_doc(document, index_list):
    """
    ---INPUT---
    document: a Doc() object processed by SpaCy Pipeline
    index_list: list of index within the document where the connectors are located
    ---OUTPUT---
    sent_units: list of Span(). Each on a sent_unit split with connectors
    """
    sent_units = []
    start_index = 0
    for end_index in index_list:
        sent_units.append(doc[start_index:end_index])
        start_index = end_index + 1
    return sent_units # list of spans


def text_to_sent_vectors(text_list, spacy_model):
    """
    Function that turns a list of text file into a dictionary of text
        sentences
    ---INPUT---
    text_list: Python list. List of text articles of length n_samples
    spacy_model: Spacy language model or pipeline. Should have text
        vector capacity
    ---OUTPUT---
    sent_dict: dictionary of structure
        {text_id:{'text': full_text,
                  'sentences': list of sentence text,
                  'sent_vectors': list of sentence vectors}}
    """
    sent_dict = {}
    for text_id, text in enumerate(text_list):
        sent_dict[text_id] = {
            'text': text,
            'sentences': [],
            'sent_vectors': []}
        for sent in spacy_model(str(text)).sents:
            sent_dict[text_id]['sentences'].append(sent.text)
            sent_dict[text_id]['sent_vectors'].append(sent.vector)
    return sent_dict

In [11]:
%%time
sent_result = text_to_sent_vectors(train_df.Text.tolist(), en_model)

CPU times: user 643 ms, sys: 0 ns, total: 643 ms
Wall time: 644 ms


In [19]:
sent_result[78]

{'text': 'Dog wasn’t a security dog - just a fluffy little guy.',
 'sentences': ['Dog wasn’t a security dog - just a fluffy little guy.'],
 'sent_vectors': [array([-1.93273470e-01,  1.48902461e-01, -1.02608308e-01, -2.78616734e-02,
          3.99077684e-02,  1.01859178e-02, -3.17826904e-02, -3.66100013e-01,
          8.91215075e-03,  2.04364681e+00, -2.08585635e-01,  3.98497656e-02,
         -1.13274612e-01, -4.36253026e-02, -2.78294623e-01, -9.19507258e-03,
          7.11109191e-02,  1.04697764e+00,  1.09406915e-02, -5.79219498e-02,
         -7.58643597e-02, -1.82325393e-01,  1.98011808e-02, -1.01253800e-02,
          5.57064377e-02, -6.62606880e-02, -1.11258470e-01, -2.16679439e-01,
          1.53960228e-01, -1.94274157e-01, -1.79098055e-01,  1.39851302e-01,
          1.19622536e-02,  1.33709997e-01,  2.12303236e-01, -4.51848507e-02,
          1.64110482e-01, -1.18232079e-01, -1.06076770e-01, -7.70037621e-02,
          1.93682164e-01,  1.25331387e-01,  4.18022349e-02, -1.82221472e-01

In [17]:
identify_connectors(en_model(train_df.Text[23]))

{'text': ['.'], 'pos': ['PUNCT'], 'index': [17]}

The sentence split does not appear to provide too much extra information here. Will pass on incorporating for now.